<a href="https://colab.research.google.com/github/Jihaad2021/Practical_PyTorch/blob/main/01_QuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
## Working with data
import torch 
from torch import nn 
from torch.utils.data import DataLoader 
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True, 
    download=True, 
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:03<00:00, 7099298.86it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 57942.30it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2217335.64it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 10600037.80it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [6]:
# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False, 
    download=True, 
    transform=ToTensor(),
)

In [7]:
batch_size = 64 

# creating data loaders
train_dataloader =DataLoader(training_data, batch_size=batch_size) 
test_dataloader =DataLoader(test_data, batch_size=batch_size) 

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break
  

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
## Creating Models 

In [10]:
# Get cpu, gpu ir mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [12]:
# Define Model 
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
## Optimizing the Model Parameters 
# to train model we need a loss function and an optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) 


In [14]:
# In a single training loop, the model makes predictions on the training dataset, and backpropagates
# the prediction error to adjust the model's parameter

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device) 

    # compute prediction error 
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [17]:
# we also check the models preformance againts the test datset to ensure it is learning
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
# The training process is conducted over several iterations (epochs)
# During each epoch the model learn parameter to make better predictions
# We print the model accuracy and loss epoch
# we'd like to see the accuracy increase and the loss decrease with every epoch 
epochs = 5 
for t in range(epochs):
  print(f"Epoch {t+1}\n-----------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn) 
print("Done!") 

Epoch 1
-----------------
loss: 2.169454 [   64/60000]
loss: 2.155320 [ 6464/60000]
loss: 2.112378 [12864/60000]
loss: 2.132657 [19264/60000]
loss: 2.069037 [25664/60000]
loss: 2.018149 [32064/60000]
loss: 2.054415 [38464/60000]
loss: 1.979571 [44864/60000]
loss: 1.981513 [51264/60000]
loss: 1.911620 [57664/60000]
Test Error: 
 Accuracy: 54.8%, Avg loss: 1.908973 

Epoch 2
-----------------
loss: 1.939980 [   64/60000]
loss: 1.901160 [ 6464/60000]
loss: 1.801827 [12864/60000]
loss: 1.844767 [19264/60000]
loss: 1.713524 [25664/60000]
loss: 1.680834 [32064/60000]
loss: 1.705090 [38464/60000]
loss: 1.614334 [44864/60000]
loss: 1.634528 [51264/60000]
loss: 1.520567 [57664/60000]
Test Error: 
 Accuracy: 61.2%, Avg loss: 1.540060 

Epoch 3
-----------------
loss: 1.610733 [   64/60000]
loss: 1.561128 [ 6464/60000]
loss: 1.425253 [12864/60000]
loss: 1.490495 [19264/60000]
loss: 1.357519 [25664/60000]
loss: 1.370349 [32064/60000]
loss: 1.378556 [38464/60000]
loss: 1.313768 [44864/60000]
loss: 

In [19]:
## Saving Models 
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [20]:
## Loading Models 
# the process for loading includes re-creating the model structure and loading the state dictionary into it
model = NeuralNetwork().to(device) 
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [21]:
# This model can now be used to make predictions
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')



Predicted: "Ankle boot", Actual: "Ankle boot"
